In [1]:
import tensorflow
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Embedding, LSTM, Dense
import numpy as np


In [2]:
inputs = ['hello', 'how are you', 'goodbye']
output = ['ohe', 'kemon acho tumi', 'biday']

outputs=['{} <eos>'.format(x) for x in output ]
outputs_i=['<sos> {}'.format(x) for x in output ]
inputs,outputs,outputs_i

(['hello', 'how are you', 'goodbye'],
 ['ohe <eos>', 'kemon acho tumi <eos>', 'biday <eos>'],
 ['<sos> ohe', '<sos> kemon acho tumi', '<sos> biday'])

In [3]:
outputs,outputs_i

(['ohe <eos>', 'kemon acho tumi <eos>', 'biday <eos>'],
 ['<sos> ohe', '<sos> kemon acho tumi', '<sos> biday'])

In [4]:
from keras.preprocessing.sequence import pad_sequences
from keras.preprocessing.text import Tokenizer



In [5]:
from keras.preprocessing.text import Tokenizer

MAX_NUM_WORDS=8

input_tokenizer = Tokenizer(num_words=MAX_NUM_WORDS)
input_tokenizer.fit_on_texts(inputs)


#
inputs_seq = input_tokenizer.texts_to_sequences(inputs)
#

#
inputs_word2index = input_tokenizer.word_index

# {a:1,go:40,here:100...........}

print('Total unique words in input:', len(inputs_word2index))
#


inputs_numwords = len(inputs_word2index)+1
#
inputs_maxlen = max(len(s) for s in inputs_seq)
print('Length of longest sentence in input:', inputs_maxlen)


# related to output!!!!!!!!!!!!>>>>>>>>>>>>>>>>>>


output_tokenizer = Tokenizer(num_words=MAX_NUM_WORDS, filters='')
output_tokenizer.fit_on_texts(outputs_i + outputs)

outputs_i_seq = output_tokenizer.texts_to_sequences(outputs_i)
outputs_seq = output_tokenizer.texts_to_sequences(outputs)



outputs_word2index = output_tokenizer.word_index
print('Total unique words in output:', len(outputs_word2index))

outputs_numwords = len(outputs_word2index)+1

outputs_maxlen = max(len(s) for s in outputs_seq)+1
print('Length of longest sentence in output:', outputs_maxlen)

Total unique words in input: 5
Length of longest sentence in input: 3
Total unique words in output: 7
Length of longest sentence in output: 5


In [6]:
outputs_word2index

{'<sos>': 1,
 '<eos>': 2,
 'ohe': 3,
 'kemon': 4,
 'acho': 5,
 'tumi': 6,
 'biday': 7}

In [7]:
output_tokenizer = Tokenizer(num_words=MAX_NUM_WORDS, filters='')
output_tokenizer.fit_on_texts(outputs_i + outputs)

In [8]:
Oidx2wrd=output_tokenizer.index_word
Owrd2idx={v:k for k,v in Oidx2wrd.items()}

# Iidx2wrd=input_tokenizer.index_word
# Iwrd2idx={v:k for k,v in Iidx2wrd.items()}



In [9]:
input_tokenizer.word_index

{'hello': 1, 'how': 2, 'are': 3, 'you': 4, 'goodbye': 5}

In [10]:
from keras.preprocessing.sequence import pad_sequences

encoder_input_sequences = pad_sequences(inputs_seq, maxlen=inputs_maxlen)
print('encoder_input_sequences shape:', encoder_input_sequences.shape)

decoder_input_sequences = pad_sequences(outputs_i_seq, maxlen=outputs_maxlen, padding='post')
print('decoder_inputs_sequences shape:', decoder_input_sequences.shape)

decoder_output_sequences = pad_sequences(outputs_seq, maxlen=outputs_maxlen, padding='post')
print('decoder_output_sequences shape:', decoder_output_sequences.shape)

encoder_input_sequences shape: (3, 3)
decoder_inputs_sequences shape: (3, 5)
decoder_output_sequences shape: (3, 5)


In [11]:
encoder_input_sequences

array([[0, 0, 1],
       [2, 3, 4],
       [0, 0, 5]])

In [12]:
decoder_input_sequences

array([[1, 3, 0, 0, 0],
       [1, 4, 5, 6, 0],
       [1, 7, 0, 0, 0]])

## the_model_cometh

In [13]:
NUM_SENTENCES = 4 # Use only the first 20,000 records.
MAX_NUM_WORDS = 4 # Use 20,000 words for tokenizing
MAX_SENT_LEN = 6

EMBEDDING_SIZE = 5

LSTM_NEURONS = 2

BATCH_SIZE = 1
EPOCHS = 3

In [14]:
from keras.layers import Input, Embedding, LSTM, Dense

encoder_embedding_layer = Embedding(
    inputs_numwords, #super important
    EMBEDDING_SIZE
    )

decoder_embedding_layer = Embedding(
    outputs_numwords, #super important
    EMBEDDING_SIZE
    )

In [15]:
encoder_inputs = Input(shape=(inputs_maxlen,))
encoder_inputs_emb = encoder_embedding_layer(encoder_inputs)
encoder = LSTM(LSTM_NEURONS, return_state=True)
encoder_outputs, h, c = encoder(encoder_inputs_emb)
encoder_states = [h, c]

decoder_inputs = Input(shape=(outputs_maxlen,))
decoder_inputs_emb = decoder_embedding_layer(decoder_inputs)
decoder = LSTM(LSTM_NEURONS, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder(decoder_inputs_emb, initial_state=encoder_states)

output_dense_layer = Dense(outputs_numwords, activation='softmax')
outputs = output_dense_layer(decoder_outputs)

In [16]:
from keras.models import Model

# defined [input] & output >>>>>>>>>>>>>>>
model = Model([encoder_inputs, decoder_inputs], outputs)


model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])
print(model.summary())


Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, 3)]                  0         []                            
                                                                                                  
 input_2 (InputLayer)        [(None, 5)]                  0         []                            
                                                                                                  
 embedding (Embedding)       (None, 3, 5)                 30        ['input_1[0][0]']             
                                                                                                  
 embedding_1 (Embedding)     (None, 5, 5)                 40        ['input_2[0][0]']             
                                                                                             

In [17]:
import tensorflow as tf
decoder_outputs_onehot  = tf.one_hot(decoder_output_sequences,

depth=outputs_numwords #this param is important!!!

)

In [18]:
decoder_outputs_onehot,decoder_outputs_onehot[0],decoder_outputs_onehot.shape

(<tf.Tensor: shape=(3, 5, 8), dtype=float32, numpy=
 array([[[0., 0., 0., 1., 0., 0., 0., 0.],
         [0., 0., 1., 0., 0., 0., 0., 0.],
         [1., 0., 0., 0., 0., 0., 0., 0.],
         [1., 0., 0., 0., 0., 0., 0., 0.],
         [1., 0., 0., 0., 0., 0., 0., 0.]],
 
        [[0., 0., 0., 0., 1., 0., 0., 0.],
         [0., 0., 0., 0., 0., 1., 0., 0.],
         [0., 0., 0., 0., 0., 0., 1., 0.],
         [0., 0., 1., 0., 0., 0., 0., 0.],
         [1., 0., 0., 0., 0., 0., 0., 0.]],
 
        [[0., 0., 0., 0., 0., 0., 0., 1.],
         [0., 0., 1., 0., 0., 0., 0., 0.],
         [1., 0., 0., 0., 0., 0., 0., 0.],
         [1., 0., 0., 0., 0., 0., 0., 0.],
         [1., 0., 0., 0., 0., 0., 0., 0.]]], dtype=float32)>,
 <tf.Tensor: shape=(5, 8), dtype=float32, numpy=
 array([[0., 0., 0., 1., 0., 0., 0., 0.],
        [0., 0., 1., 0., 0., 0., 0., 0.],
        [1., 0., 0., 0., 0., 0., 0., 0.],
        [1., 0., 0., 0., 0., 0., 0., 0.],
        [1., 0., 0., 0., 0., 0., 0., 0.]], dtype=float32)>,
 

In [19]:
trn = model.fit([encoder_input_sequences, decoder_input_sequences],

               decoder_outputs_onehot, epochs=1000
               )





Epoch 1/1000


1/1 [==============================] - 6s 6s/step - loss: 2.0806 - accuracy: 0.0667
Epoch 2/1000
1/1 [==============================] - 0s 17ms/step - loss: 2.0754 - accuracy: 0.2667
Epoch 3/1000
1/1 [==============================] - 0s 16ms/step - loss: 2.0716 - accuracy: 0.2667
Epoch 4/1000
1/1 [==============================] - 0s 17ms/step - loss: 2.0684 - accuracy: 0.4000
Epoch 5/1000
1/1 [==============================] - 0s 17ms/step - loss: 2.0655 - accuracy: 0.4667
Epoch 6/1000
1/1 [==============================] - 0s 16ms/step - loss: 2.0628 - accuracy: 0.4667
Epoch 7/1000
1/1 [==============================] - 0s 17ms/step - loss: 2.0602 - accuracy: 0.4667
Epoch 8/1000
1/1 [==============================] - 0s 17ms/step - loss: 2.0578 - accuracy: 0.4667
Epoch 9/1000
1/1 [==============================] - 0s 17ms/step - loss: 2.0555 - accuracy: 0.4667
Epoch 10/1000
1/1 [==============================] - 0s 0s/step - loss: 2.0532 - accuracy: 0.4667
Epoch 11/10

In [20]:
# Iidx2wrd,Oidx2wrd,
# output_tokenizer.word_index

## THE_INFERENCE `is here boiis`


In [21]:
input_seq='how you hello'
input_seq=input_tokenizer.texts_to_sequences([input_seq])
input_seq

[[2, 4, 1]]

In [22]:
encoder_model = Model(encoder_inputs, encoder_states)
print(encoder_model.summary())

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 3)]               0         
                                                                 
 embedding (Embedding)       (None, 3, 5)              30        
                                                                 
 lstm (LSTM)                 [(None, 2),               64        
                              (None, 2),                         
                              (None, 2)]                         
                                                                 
Total params: 94 (376.00 Byte)
Trainable params: 94 (376.00 Byte)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
None


In [23]:
decoder_input_h = Input(shape=(LSTM_NEURONS,))
decoder_input_c = Input(shape=(LSTM_NEURONS,))
decoder_input_states = [decoder_input_h, decoder_input_c]

decoder_input_word = Input(shape=(1,))
decoder_input_word_emb = decoder_embedding_layer(decoder_input_word)

decoder_outputs, h, c = decoder(decoder_input_word_emb, initial_state=decoder_input_states)
decoder_states = [h, c]

outputs = output_dense_layer(decoder_outputs)
decoder_model = Model([decoder_input_word]+decoder_input_states, [outputs]+decoder_states)
print(decoder_model.summary())

Model: "model_2"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_5 (InputLayer)        [(None, 1)]                  0         []                            
                                                                                                  
 embedding_1 (Embedding)     multiple                     40        ['input_5[0][0]']             
                                                                                                  
 input_3 (InputLayer)        [(None, 2)]                  0         []                            
                                                                                                  
 input_4 (InputLayer)        [(None, 2)]                  0         []                            
                                                                                            

In [24]:
states = encoder_model.predict(input_seq)

1/1 [==============================] - 1s 616ms/step


In [25]:
states

[array([[ 0.22516811, -0.6741042 ]], dtype=float32),
 array([[ 1.3700013, -1.3576834]], dtype=float32)]

In [26]:
sos= outputs_word2index['<sos>']
eos = outputs_word2index['<eos>']
sos,eos

(1, 2)

In [27]:
import numpy as np
from numpy import zeros,argmax,argsort

output_seq = zeros((1, 1))
output_seq[0, 0] = sos
output_seq

array([[1.]])

In [28]:
# after running all other
# states = [h, c]
# output_seq[0, 0] = idx

In [29]:
output_tokens, h, c = decoder_model.predict([output_seq]+states)

1/1 [==============================] - 1s 793ms/step


In [30]:
output_tokens.shape ,output_tokens[0,0,:] , output_tokens

((1, 1, 8),
 array([0.1077996 , 0.04039694, 0.0137523 , 0.01786779, 0.48442328,
        0.26857722, 0.0555376 , 0.01164523], dtype=float32),
 array([[[0.1077996 , 0.04039694, 0.0137523 , 0.01786779, 0.48442328,
          0.26857722, 0.0555376 , 0.01164523]]], dtype=float32))

In [31]:
print(h)
print('\n\n')
print(c)
print(Oidx2wrd)

[[ 0.8588016  -0.72210854]]



[[ 1.3478651  -0.95389736]]
{1: '<sos>', 2: '<eos>', 3: 'ohe', 4: 'kemon', 5: 'acho', 6: 'tumi', 7: 'biday'}


### how are you

In [32]:
output_tokens
idx = argmax(output_tokens)
idx
# word = Oidx2wrd[idx]
# word,idx

4

In [33]:
word='<sos>'
input_seq='how are you'
input_seq=input_tokenizer.texts_to_sequences([input_seq])
states = encoder_model.predict(input_seq)
while word!='<eos>':

  output_tokens, h, c = decoder_model.predict([output_seq]+states)
  print(output_tokens)
  idx,srt = argmax(output_tokens,),np.argsort(output_tokens)

  output_seq[0, 0] = idx
  states=[h,c]
  word = Oidx2wrd[idx]

  print(word,idx,srt)

1/1 [==============================] - 0s 88ms/step


1/1 [==============================] - 0s 51ms/step
[[[0.09054932 0.03386606 0.00755689 0.01148068 0.56718934 0.2452295
   0.03712631 0.00700186]]]
kemon 4 [[[7 2 3 1 6 0 5 4]]]
1/1 [==============================] - 0s 51ms/step
[[[0.03446657 0.04119627 0.03386435 0.02696405 0.15020889 0.3896563
   0.30039018 0.02325344]]]
acho 5 [[[7 3 2 0 1 4 6 5]]]
1/1 [==============================] - 0s 89ms/step
[[[0.01020189 0.02485483 0.04212608 0.0232145  0.0377858  0.28122532
   0.55612713 0.02446449]]]
tumi 6 [[[0 3 7 1 4 2 5 6]]]
1/1 [==============================] - 0s 27ms/step
[[[0.14459576 0.01365746 0.47367007 0.1353451  0.00768285 0.01874963
   0.0525902  0.1537089 ]]]
<eos> 2 [[[4 1 5 6 3 0 7 2]]]


In [34]:
decoder_input = np.zeros((1, 1))
decoder_input[0, 0] = sos

sequences = [[decoder_input, 0.0]]
sequences

[[array([[1.]]), 0.0]]

In [ ]:

def BeamSearch(states,decoder_model,outputs_maxlen,beam_size=2):
  max_len=outputs_maxlen+1
  sos=Oidx2wrd[1]
  decoder_input = np.zeros((1, 1))
  decoder_input[0, 0] = 1

  sequences = [[decoder_input, 0.0]]

  ln=sequences[0][0][0]

  neWeight={0:states}
  ii=0
  while len(ln) < max_len:
      all_candidates = []



      for i in range(len(sequences)):
          seq, score = sequences[i]


          decoder_input[0,0]=seq[0][-1]
          if ii!=0:
            current_states = states[i]
          else:current_states = states
          # print('input >>>',seq, decoder_input,)

          probs, h, c = decoder_model.predict([decoder_input]+current_states)
          len(probs)
          word_preds = np.argsort(probs)[0][0][-beam_size:]


          for w in word_preds:
              candidate_seq = [
                  np.concatenate([seq, np.array([[w]])], axis=1),

                              score
                              - np.log(probs[-1][-1][w])]
              new_states = [h, c]
              all_candidates.append((candidate_seq, new_states))
      ii+=1
      all_candidates.sort(key=lambda tup: tup[0][1])
      sequences = [tup[0] for tup in all_candidates]
      states = [tup[1] for tup in all_candidates]



      ln=(sequences[0][0][0])
  min_array = min(sequences, key=lambda x: x[1])
  return min_array[0],sequences


{1: '<sos>', 2: '<eos>', 3: 'ohe', 4: 'kemon', 5: 'acho', 6: 'tumi', 7: 'biday'}
({1: 'hello', 2: 'how', 3: 'are', 4: 'you', 5: 'goodbye'},

In [ ]:
input_seq='how are you'
input_seq=input_tokenizer.texts_to_sequences([input_seq])
input_seq=[[2,3,4]]
states = encoder_model.predict(input_seq)
x,seq=BeamSearch(states,decoder_model ,outputs_maxlen,beam_size=2)

1/1 [==============================] - 0s 24ms/step


In [ ]:
seq

[[array([[1., 4., 5., 6., 2., 2.]]), 4.441096544265747],
 [array([[1., 4., 6., 3., 0., 0.]]), 4.769274711608887],
 [array([[1., 4., 5., 5., 6., 2.]]), 4.927106618881226],
 [array([[1., 4., 5., 6., 3., 2.]]), 5.052044034004211],
 [array([[1., 4., 5., 6., 2., 3.]]), 5.230000138282776],
 [array([[1., 4., 6., 2., 2., 2.]]), 5.4529993534088135],
 [array([[1., 5., 4., 5., 6., 2.]]), 5.554852485656738],
 [array([[1., 4., 6., 3., 2., 0.]]), 5.582513928413391],
 [array([[1., 4., 6., 2., 3., 0.]]), 5.734493017196655],
 [array([[1., 5., 4., 6., 2., 2.]]), 5.746296763420105],
 [array([[1., 4., 5., 5., 2., 2.]]), 5.808568358421326],
 [array([[1., 4., 5., 5., 2., 6.]]), 5.848628520965576],
 [array([[1., 4., 5., 5., 6., 3.]]), 5.94578230381012],
 [array([[1., 4., 5., 6., 3., 0.]]), 6.069006562232971],
 [array([[1., 4., 6., 2., 3., 2.]]), 6.072152376174927],
 [array([[1., 4., 6., 2., 2., 3.]]), 6.125104665756226],
 [array([[1., 5., 5., 6., 2., 2.]]), 6.136052429676056],
 [array([[1., 5., 5., 6., 0., 0

In [ ]:
# min_array = min(sequences, key=lambda x: x[1])
# x=min_array[0]
x

array([[1., 4., 5., 6., 2., 2.]])

In [ ]:

for i in x[0]:
    print(i,Oidx2wrd[i])

1.0 <sos>
4.0 kemon
5.0 acho
6.0 tumi
2.0 <eos>
2.0 <eos>


In [ ]:
start_seq = np.array([[1]])
initial_states = [states] + [np.zeros((1, LSTM_NEURONS))] * 2  #


# TRANSFORMERS

In [81]:
text_pairs=[]
# [('He turned to look Langdon in the eye.',
#   '[start] সে ঘুরে ল্যাংডনের চোখের দিকে তাকায়। [end]'),

en_lst = ['hello', 'how are you', 'goodbye']
ben_lst = ['ohe', 'kemon acho tumi', 'biday']
len(ben_lst),len(en_lst)



(3, 3)

In [82]:
text_pairs=[]
cnt=0
for eng,ben in zip(en_lst,ben_lst):
  if len(ben.split())>50:
    continue
  ben = "[start] " + ben + " [end]"
  text_pairs.append((eng,ben))
  cnt+=1
  if cnt>=(2753069//3):breaka
len(text_pairs)

text_pairs


[('hello', '[start] ohe [end]'),
 ('how are you', '[start] kemon acho tumi [end]'),
 ('goodbye', '[start] biday [end]')]

In [83]:
import tensorflow as tf
import re

# Define the characters to keep
keep_chars = ".,?!| / -"

def custom_standardization(input_string):
    # Convert to lowercase
    lowercase = tf.strings.lower(input_string)

    # Remove non-alphabetic characters, except for those in the keep_chars set and digits
    stripped = tf.strings.regex_replace(lowercase, "[^a-z0-9" + re.escape(keep_chars) + "]", "")

    return stripped

# Example usage:
strip_chars = "!@#$%^&*()_-+=[]{}|;:',.<>?/"
input_text = tf.constant("Hello, World! How are you doing? 12/3")
processed_text = custom_standardization(input_text)

print(processed_text.numpy().decode("utf-8"))


hello, world! how are you doing? 12/3


In [84]:
import random
random.shuffle(text_pairs)
num_val_samples = int(0* len(text_pairs))
num_train_samples = len(text_pairs) - 2 * num_val_samples
train_pairs = text_pairs[:num_train_samples]
val_pairs = text_pairs[num_train_samples:num_train_samples + num_val_samples]
test_pairs = text_pairs[num_train_samples + num_val_samples:]

In [85]:
text_pairs

[('how are you', '[start] kemon acho tumi [end]'),
 ('hello', '[start] ohe [end]'),
 ('goodbye', '[start] biday [end]')]

In [86]:
import tensorflow as tf
import string
import re
from tensorflow.keras import layers
sequence_length=5
max_token=8
source_vectorization = layers.TextVectorization(
  max_tokens=max_token,
  output_mode="int",
  output_sequence_length=sequence_length,
  standardize=custom_standardization
)
target_vectorization = layers.TextVectorization(
  max_tokens=max_token,
  output_mode="int",
  output_sequence_length=sequence_length + 1,

)

In [87]:

train_eng_texts = [pair[0] for pair in train_pairs]
train_ben_texts = [pair[1] for pair in train_pairs]

train_eng_texts,train_ben_texts


(['how are you', 'hello', 'goodbye'],
 ['[start] kemon acho tumi [end]', '[start] ohe [end]', '[start] biday [end]'])

In [88]:
def createVector(pairs,vectorztn):
  batch_size = 10000  # Adjust the batch size based on your available memory
  num_batches = len(pairs) // batch_size + 1

  for i in range(num_batches):
      start_idx = i * batch_size
      end_idx = min((i + 1) * batch_size, len(pairs))
      batch_texts = pairs[start_idx:end_idx]
      vectorztn.adapt(batch_texts)
      # target_vectorization.adapt(batch_ben_texts)  # Uncomment if needed



In [92]:
train_eng_texts

['how are you', 'hello', 'goodbye']

In [89]:
createVector(train_eng_texts,source_vectorization)
createVector(train_ben_texts,target_vectorization)


In [90]:

vocabulary = source_vectorization.get_vocabulary()

# Access other relevant information
max_tokens = source_vectorization.get_config()['max_tokens']
output_sequence_length = source_vectorization.get_config()['output_sequence_length']
output_mode = source_vectorization.get_config()['output_mode']
standardization = source_vectorization.get_config()['standardize']

# Print or inspect the information
print("Vocabulary:", vocabulary)
print("Max Tokens:", max_tokens)
print("Output Sequence Length:", output_sequence_length)
print("Output Mode:", output_mode)
print("Standardization:", standardization)


Vocabulary: ['', '[UNK]', 'you', 'how', 'hello', 'goodbye', 'are']
Max Tokens: 8
Output Sequence Length: 5
Output Mode: int
Standardization: <function custom_standardization at 0x0000021B673085E0>


In [93]:
import tensorflow as tf

# Assuming source_vectorization and target_vectorization are TextVectorization layers

batch_size =1

def format_dataset(eng, ben):
    print(ben)
    eng = source_vectorization(eng)
    ben = target_vectorization(ben)
    print(ben )
    # Ensure ben has the expected shape before slicing
    # ben = tf.reshape(ben, [tf.shape(ben)[0], -1])  # Reshape to 2D
    return {"eng": eng, "ben": ben[:, :-1]}, ben[:, 1:]

def make_dataset(pairs):
    eng_texts, ben_texts = zip(*pairs)
    print(eng_texts)
    eng_texts = list(eng_texts)
    ben_texts = list(ben_texts)
    dataset = tf.data.Dataset.from_tensor_slices((eng_texts, ben_texts))
    dataset = dataset.batch(batch_size)
    dataset = dataset.map(format_dataset, num_parallel_calls=tf.data.experimental.AUTOTUNE)
    return dataset.shuffle(2048).prefetch(16).cache()


In [95]:
train_pairs

[('how are you', '[start] kemon acho tumi [end]'),
 ('hello', '[start] ohe [end]'),
 ('goodbye', '[start] biday [end]')]

In [94]:
import numpy as np
train_ds = make_dataset(train_pairs)



('how are you', 'hello', 'goodbye')
Tensor("args_1:0", shape=(None,), dtype=string)
Tensor("text_vectorization_3/RaggedToTensor/RaggedTensorToTensor:0", shape=(None, 6), dtype=int64)


## ENCODER

In [15]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

class TransformerEncoder(layers.Layer):
    def __init__(self, embed_dim, dense_dim, num_heads, **kwargs):
        super().__init__(**kwargs)
        self.embed_dim = embed_dim
        self.dense_dim = dense_dim
        self.num_heads = num_heads
        self.attention = layers.MultiHeadAttention(
            num_heads=num_heads, key_dim=embed_dim)
        self.dense_proj = keras.Sequential(
            [layers.Dense(dense_dim, activation="relu"),
             layers.Dense(embed_dim),]
        )
        self.layernorm_1 = layers.LayerNormalization()
        self.layernorm_2 = layers.LayerNormalization()

    def call(self, inputs, mask=None):
        if mask is not None:
            mask = mask[:, tf.newaxis, :]
        attention_output = self.attention(
            inputs, inputs, attention_mask=mask)
        proj_input = self.layernorm_1(inputs + attention_output)
        proj_output = self.dense_proj(proj_input)
        return self.layernorm_2(proj_input + proj_output)

    def get_config(self):
        config = super().get_config()
        config.update({
            "embed_dim": self.embed_dim,
            "num_heads": self.num_heads,
            "dense_dim": self.dense_dim,
        })
        return config

In [16]:
class TransformerDecoder(layers.Layer):
    def __init__(self, embed_dim, dense_dim, num_heads, **kwargs):
        super().__init__(**kwargs)
        self.embed_dim = embed_dim
        self.dense_dim = dense_dim
        self.num_heads = num_heads
        self.attention_1 = layers.MultiHeadAttention(
            num_heads=num_heads, key_dim=embed_dim)
        self.attention_2 = layers.MultiHeadAttention(
            num_heads=num_heads, key_dim=embed_dim)
        self.dense_proj = keras.Sequential(
            [layers.Dense(dense_dim, activation="relu"),
             layers.Dense(embed_dim),]
        )
        self.layernorm_1 = layers.LayerNormalization()
        self.layernorm_2 = layers.LayerNormalization()
        self.layernorm_3 = layers.LayerNormalization()
        self.supports_masking = True

    def get_config(self):
        config = super().get_config()
        config.update({
            "embed_dim": self.embed_dim,
            "num_heads": self.num_heads,
            "dense_dim": self.dense_dim,
        })
        return config

    def get_causal_attention_mask(self, inputs):
        input_shape = tf.shape(inputs)
        batch_size, sequence_length = input_shape[0], input_shape[1]
        i = tf.range(sequence_length)[:, tf.newaxis]
        j = tf.range(sequence_length)
        mask = tf.cast(i >= j, dtype="int32")
        mask = tf.reshape(mask, (1, input_shape[1], input_shape[1]))
        mult = tf.concat(
            [tf.expand_dims(batch_size, -1),
             tf.constant([1, 1], dtype=tf.int32)], axis=0)
        return tf.tile(mask, mult)

    def call(self, inputs, encoder_outputs, mask=None):
        causal_mask = self.get_causal_attention_mask(inputs)
        if mask is not None:
            padding_mask = tf.cast(
                mask[:, tf.newaxis, :], dtype="int32")
            padding_mask = tf.minimum(padding_mask, causal_mask)
        else:
            padding_mask = mask
        attention_output_1 = self.attention_1(
            query=inputs,
            value=inputs,
            key=inputs,
            attention_mask=causal_mask)
        attention_output_1 = self.layernorm_1(inputs + attention_output_1)
        attention_output_2 = self.attention_2(
            query=attention_output_1,
            value=encoder_outputs,
            key=encoder_outputs,
            attention_mask=padding_mask,
        )
        attention_output_2 = self.layernorm_2(
            attention_output_1 + attention_output_2)
        proj_output = self.dense_proj(attention_output_2)
        return self.layernorm_3(attention_output_2 + proj_output)

In [17]:
class PositionalEmbedding(layers.Layer):
    def __init__(self, sequence_length, input_dim, output_dim, **kwargs):
        super().__init__(**kwargs)
        self.token_embeddings = layers.Embedding(
            input_dim=input_dim, output_dim=output_dim)
        self.position_embeddings = layers.Embedding(
            input_dim=sequence_length, output_dim=output_dim)
        self.sequence_length = sequence_length
        self.input_dim = input_dim
        self.output_dim = output_dim

    def call(self, inputs):
        length = tf.shape(inputs)[-1]
        positions = tf.range(start=0, limit=length, delta=1)
        embedded_tokens = self.token_embeddings(inputs)
        embedded_positions = self.position_embeddings(positions)
        return embedded_tokens + embedded_positions

    def compute_mask(self, inputs, mask=None):
        return tf.math.not_equal(inputs, 0)

    def get_config(self):
        config = super(PositionalEmbedding, self).get_config()
        config.update({
            "output_dim": self.output_dim,
            "sequence_length": self.sequence_length,
            "input_dim": self.input_dim,
        })
        return config

## THY MODEL COMETH

In [18]:
sequence_length

5

In [19]:
embed_dim = 7
dense_dim = 5
num_heads = 3 
vocab_size=max_token
encoder_inputs = keras.Input(shape=(None,), dtype="int64", name="eng")
x = PositionalEmbedding(sequence_length, vocab_size, embed_dim)(encoder_inputs)
encoder_outputs = TransformerEncoder(embed_dim, dense_dim, num_heads)(x)

decoder_inputs = keras.Input(shape=(None,), dtype="int64", name="ben")
x = PositionalEmbedding(sequence_length, vocab_size, 5)(decoder_inputs)
x = TransformerDecoder(5, 1, 1)(x, encoder_outputs)
x = layers.Dropout(0.35)(x)
decoder_outputs = layers.Dense(vocab_size, activation="softmax")(x)
transformer = keras.Model([encoder_inputs, decoder_inputs], decoder_outputs)

In [20]:
transformer.compile(
    optimizer="rmsprop",
    loss="sparse_categorical_crossentropy",
    metrics=["accuracy"])
transformer.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 eng (InputLayer)            [(None, None)]               0         []                            
                                                                                                  
 ben (InputLayer)            [(None, None)]               0         []                            
                                                                                                  
 positional_embedding (Posi  (None, None, 7)              91        ['eng[0][0]']                 
 tionalEmbedding)                                                                                 
                                                                                                  
 positional_embedding_1 (Po  (None, None, 5)              65        ['ben[0][0]']             

In [21]:
train_ds

<CacheDataset element_spec=({'eng': TensorSpec(shape=(None, 5), dtype=tf.int64, name=None), 'ben': TensorSpec(shape=(None, 5), dtype=tf.int64, name=None)}, TensorSpec(shape=(None, 5), dtype=tf.int64, name=None))>

In [ ]:
transformer.fit(train_ds, epochs=300)

In [45]:
import numpy as np
spa_vocab = target_vectorization.get_vocabulary()
spa_index_lookup = dict(zip(range(len(spa_vocab)), spa_vocab))
max_decoded_sentence_length = 5

def decode_sequence(input_sentence):
    tokenized_input_sentence = source_vectorization([input_sentence])
    decoded_sentence = "[start]"
    for i in range(max_decoded_sentence_length):
        tokenized_target_sentence = target_vectorization(
            [decoded_sentence])[:, :-1]
        predictions = transformer(
            [tokenized_input_sentence, tokenized_target_sentence])
        # print(predictions)
        sampled_token_index = np.argmax(predictions[0, i, :])
        sampled_token = spa_index_lookup[sampled_token_index]
        decoded_sentence += " " + sampled_token
        if sampled_token == "end":
            break
    return decoded_sentence


test_pairs=train_pairs

test_eng_texts = [pair[0] for pair in test_pairs]
print(test_eng_texts)

# >>>>>>>>>>>>>>>>>
input_sentence = random.choice(test_eng_texts)
print("-")
print(input_sentence)
print(decode_sequence(input_sentence))

['hello', 'how are you', 'goodbye']
-
hello
[start] ohe end


In [29]:
import numpy as np

def pad_array(input_array, desired_length):
    if len(input_array.shape) == 1:
        input_array = np.array([input_array])
    
    current_length = input_array.shape[1]
    
    if current_length >= desired_length:
        return input_array
    
    # Calculate the number of zeros to add
    zeros_to_add = desired_length - current_length
    
    # Create a padding array with zeros
    padding = np.zeros((input_array.shape[0], zeros_to_add))
    
    # Concatenate the padding array to the input_array
    result_array = np.concatenate((input_array, padding), axis=1)
    
    return result_array

# Example usage:
array1 = np.array([[2.]])
array2 = np.array([2., 3])

result_array1 = pad_array(array1, 6)
result_array2 = pad_array(array2, 6)

print("Result Array 1:", result_array1)
print("Result Array 2:", result_array2)


Result Array 1: [[2. 0. 0. 0. 0. 0.]]
Result Array 2: [[2. 3. 0. 0. 0. 0.]]


In [39]:
spa_vocab = target_vectorization.get_vocabulary()
spa_index_lookup = dict(zip(range(len(spa_vocab)), spa_vocab))



def TBeamSearch(inputs,decoder_input,decoder_model,outputs_maxlen,beam_size=2):
  
  
  max_len=outputs_maxlen

  decoded_sentence = np.zeros((1, 1))
  decoded_sentence[0, 0] = 2
 

  sequences = [[decoded_sentence, 0.0]]


  decoded_sentence = "[start]"
  tokenized_target_sentence = target_vectorization([decoded_sentence])[:, :-1]

 
  



  ln=sequences[0][0][0]
  print('>>>>>>>>>>',ln)
 
  ii=0
  while ii < max_len:
      all_candidates = []



      for i in range(len(sequences)):
          seq, score = sequences[i]
          
          tokenized_target_sentence=pad_array(seq,max_len)

 

          probs = transformer(
            [inputs, tokenized_target_sentence])
          word_preds = np.argsort(probs)[0][ii][-beam_size:]


          for w in word_preds:
              candidate_seq = [
                  # np.concatenate([seq, np.array([w])], axis=1),
                  np.append(seq, w),
                      score - np.log(probs[-1][-1][w])]
                      
              new_states = w
          
              all_candidates.append((candidate_seq, new_states))
      ii+=1
      all_candidates.sort(key=lambda tup: tup[0][1])
      sequences = [tup[0] for tup in all_candidates]
      states = [tup[1] for tup in all_candidates]



      ln=(sequences[0][0][0])
  min_array = min(sequences, key=lambda x: x[1])
  return min_array[0],sequences




test_eng_texts = [pair[0] for pair in test_pairs]
print(test_eng_texts)

input_sentence = random.choice(test_eng_texts)
print(input_sentence)
tokenized_input_sentence = source_vectorization([input_sentence])
# input >>>>>>>>>>>>>>>>>>


decoded_sentence = "[start]"

tokenized_target_sentence = target_vectorization([decoded_sentence])[:, :-1]

tokenized_target_sentence

x,y=TBeamSearch(tokenized_input_sentence , tokenized_target_sentence , transformer , 5)
print(y)

for xx in x:
  print(xx,spa_index_lookup[xx])


['hello', 'how are you', 'goodbye']
hello
>>>>>>>>>> [2.]
[[array([2., 6., 4., 0., 0., 0.]), 16.171515756868757], [array([2., 5., 5., 0., 0., 0.]), 18.846020584227517], [array([2., 5., 3., 0., 0., 0.]), 20.474569075973704], [array([2., 6., 4., 0., 0., 4.]), 23.64329182705842], [array([2., 6., 1., 0., 5., 3.]), 25.040064440574497], [array([2., 5., 5., 0., 0., 4.]), 26.077192548196763], [array([2., 6., 4., 0., 3., 0.]), 27.426995853427798], [array([2., 6., 1., 0., 5., 0.]), 27.75573524227366], [array([2., 6., 1., 0., 3., 0.]), 27.888044032617472], [array([2., 5., 3., 0., 0., 4.]), 28.0618867832236], [array([2., 5., 5., 0., 3., 0.]), 30.362288037664257], [array([2., 5., 5., 3., 0., 0.]), 30.362640558509156], [array([2., 5., 3., 0., 3., 0.]), 32.176250254153274], [array([2., 6., 1., 4., 5., 3.]), 32.271445982158184], [array([2., 6., 4., 4., 5., 3.]), 32.37278962135315], [array([2., 6., 4., 4., 5., 0.]), 33.69346046447754], [array([2., 6., 4., 4., 3., 0.]), 34.39391063223593], [array([2., 6

array([2., 6., 7., 0., 1., 4.])